In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
HISTORY = 6 #indicates how many past datapoints are considered (e.g prediction based on pas 6 hours)
FUTURE = 1  #indicates prediction into future (e.g 1 hour into the future)
OVERLAP = 1 #indicates how much the data is overlapped

In [3]:
data = pd.read_csv("../1h_sampled_data.csv")

In [4]:
data

,time,temp_pi,hum_pi,PM1_pi,PM2.5_pi,PM10_pi,temp_blue,precip,hum_blue,wind_speed,...,hum_sbs,wind,pressure,PM2.5,PM10,AQI,sunrise,noon,sunset,daylight
0,2021-03-23 10:00:00+01:00,8.316667,53.283333,6.042500,11.798333,17.710000,3.106314,0.0,76.0,5.116561,...,65.0,3.7,1025.0,16.0,24.3,59.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414
1,2021-03-23 11:00:00+01:00,13.408333,38.358333,4.840833,10.010000,15.388333,5.416314,0.0,67.0,6.924738,...,65.0,3.7,1025.0,14.0,20.9,55.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414
2,2021-03-23 12:00:00+01:00,17.391667,32.191667,5.140833,7.510833,9.632500,7.216313,0.0,55.0,7.421590,...,57.0,14.8,1025.0,12.0,17.3,50.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414
3,2021-03-23 13:00:00+01:00,18.650000,30.083333,3.752500,6.064167,8.284167,8.656314,0.0,48.0,6.952755,...,57.0,14.8,1025.0,NaN,NaN,27.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414
4,2021-03-23 14:00:00+01:00,19.983333,28.125000,4.610000,10.042500,15.750000,9.586313,0.0,39.0,11.269782,...,40.0,11.1,1025.0,11.0,16.2,45.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,2021-04-12 10:00:00+02:00,5.350000,77.241667,3.190000,6.740000,10.450833,1.286314,1.2,93.0,16.935310,...,93.0,5.5,1023.0,3.0,5.6,12.0,2021-04-12 06:43:17.148768+02:00,2021-04-12 13:26:40+02:00,2021-04-12 20:10:43.726239+02:00,0 days 13:27:26.577471
481,2021-04-12 11:00:00+02:00,5.975000,75.625000,1.982500,3.617500,5.262500,1.306313,0.9,92.0,17.917007,...,93.0,9.3,1024.0,NaN,5.8,10.0,2021-04-12 06:43:17.148768+02:00,2021-04-12 13:26:40+02:00,2021-04-12 20:10:43.726239+02:00,0 days 13:27:26.577471
482,2021-04-12 12:00:00+02:00,6.372727,72.300000,3.828182,8.692727,13.842727,1.426314,0.8,92.0,13.684735,...,93.0,9.3,1024.0,NaN,6.0,11.0,2021-04-12 06:43:17.148768+02:00,2021-04-12 13:26:40+02:00,2021-04-12 20:10:43.726239+02:00,0 days 13:27:26.577471
483,2021-04-12 13:00:00+02:00,6.641667,71.333333,4.800000,9.065833,13.411667,2.436314,0.4,83.0,13.608762,...,93.0,1.8,1026.0,NaN,6.0,13.0,2021-04-12 06:43:17.148768+02:00,2021-04-12 13:26:40+02:00,2021-04-12 20:10:43.726239+02:00,0 days 13:27:26.577471


In [5]:
hours = pd.to_datetime(data['time'], utc=True).dt.hour

In [6]:
features = ['temp_pi', 'hum_pi', 'PM1_pi', 'PM2.5_pi', 'PM10_pi', 'wind_speed']

In [7]:
targets = ['PM2.5_pi', 'PM10_pi']

In [8]:
data = data[features]

In [9]:
features_new = []
for i in range(HISTORY):
    features_new += [f + "_t" + str(i) for f in features]

In [10]:
features_new += targets 
features_new += ['hour']

In [11]:
x = math.ceil(data.shape[0]/OVERLAP)
y = len(features_new) 
data_new = np.zeros((x, y))

In [12]:
k = 0
for i in range(HISTORY - 1, data.shape[0] - 1, OVERLAP):
    temp = np.zeros(len(features_new))
    for j in range(HISTORY):
        temp[j*(HISTORY):(j+1)*(HISTORY)] = data.iloc[i - j, :]
    
    temp[HISTORY*HISTORY] = data.iloc[i+1, 3]
    temp[HISTORY*HISTORY+1] = data.iloc[i+1, 4]
    temp[HISTORY*HISTORY+2] = hours[i]
    data_new[k,:] = temp
    k += 1

In [13]:
data_new = pd.DataFrame(data_new, columns=features_new)

In [14]:
data_new

,temp_pi_t0,hum_pi_t0,PM1_pi_t0,PM2.5_pi_t0,PM10_pi_t0,wind_speed_t0,temp_pi_t1,hum_pi_t1,PM1_pi_t1,PM2.5_pi_t1,...,wind_speed_t4,temp_pi_t5,hum_pi_t5,PM1_pi_t5,PM2.5_pi_t5,PM10_pi_t5,wind_speed_t5,PM2.5_pi,PM10_pi,hour
0,17.625000,30.933333,4.365000,9.6525,15.223333,9.826088,19.983333,28.125000,4.610000,10.0425,...,6.924738,8.316667,53.283333,6.042500,11.798333,17.710000,5.116561,6.641000,9.477000,14.0
1,13.990000,35.770000,3.787000,6.6410,9.477000,8.209263,17.625000,30.933333,4.365000,9.6525,...,7.421590,13.408333,38.358333,4.840833,10.010000,15.388333,6.924738,10.465000,16.762500,15.0
2,12.683333,37.733333,4.529167,10.4650,16.762500,8.496304,13.990000,35.770000,3.787000,6.6410,...,6.952755,17.391667,32.191667,5.140833,7.510833,9.632500,7.421590,6.040000,9.553333,16.0
3,11.616667,40.350000,2.710000,6.0400,9.553333,7.754637,12.683333,37.733333,4.529167,10.4650,...,11.269782,18.650000,30.083333,3.752500,6.064167,8.284167,6.952755,7.147500,10.535000,17.0
4,10.425000,42.900000,3.815000,7.1475,10.535000,6.162207,11.616667,40.350000,2.710000,6.0400,...,9.826088,19.983333,28.125000,4.610000,10.042500,15.750000,11.269782,7.878333,12.093333,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
481,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
482,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
483,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [15]:
data_new.to_csv('model_data_pi_hours.csv')